In [1]:
from evalscope.run import run_task
import modeling_tinyllm

task_cfg = {
    'model': 'learn2pro/buddygpt-0.2b-base-zh',
    'datasets': ['cmmlu'],
    "eval-batch-size": 32,
}

run_task(task_cfg=task_cfg)

[2025-05-27 16:57:14,976] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/samtang/miniconda3/envs/rl/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/samtang/miniconda3/envs/rl

2025-05-27 16:57:16,968 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2025-05-27 16:57:17,372 - modelscope - INFO - Target directory already exists, skipping creation.


2025-05-27 16:57:17,815 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2025-05-27 16:57:18,070 - modelscope - INFO - Target directory already exists, skipping creation.
TinyllmForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, pl

ImportError: attempted relative import with no known parent package

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-0.6B' ,trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
len(tokenizer)

151669

In [4]:
import torch

(torch.arange(0, 10, 2) / 10).dtype

torch.float32

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import modeling_tinyllm
from pretrain import print_parameters

model_id = 'outputs/buddygpt-qwen3'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
print_parameters(model)
model.to('cuda')


[2025-05-27 15:42:00,413] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/samtang/miniconda3/envs/rl/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/samtang/miniconda3/envs/rl/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/samtang/miniconda3/envs/rl

total param 244.17333984375m


TinyllmForCausalLM(
  (model): TinyllmModel(
    (embed_tokens): Embedding(151669, 512)
    (layers): ModuleList(
      (0-23): 24 x TinyllmDecoderLayer(
        (self_attn): TinyllmSdpaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=True)
          (k_proj): Linear(in_features=512, out_features=512, bias=True)
          (v_proj): Linear(in_features=512, out_features=512, bias=True)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): TinyllmRotaryEmbedding()
        )
        (mlp): TinyllmMLP(
          (gate_proj): Linear(in_features=512, out_features=2048, bias=False)
          (up_proj): Linear(in_features=512, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): TinyllmRMSNorm()
        (post_attention_layernorm): TinyllmRMSNorm()
      )
    )
    (norm): TinyllmRMSNorm()
  )
  (lm_head)

In [2]:
def sample(model, query, max_length=128):
    input_ids = tokenizer.encode(query, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids=input_ids,
        max_new_tokens=max_length,
    )
    gen_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return gen_text

for _ in range(10):
    # res = sample(model, 'which is the capital of China?')
    res = sample(model, '中国首都是哪?')
    print(res)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


中国首都是哪?中国在2015年12月15日发布《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国
中国首都是哪?中国在2015年12月15日发布《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国
中国首都是哪?中国在2015年12月15日发布《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国
中国首都是哪?中国在2015年12月15日发布《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国
中国首都是哪?中国在2015年12月15日发布《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国
中国首都是哪?中国在2015年12月15日发布《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国
中国首都是哪?中国在2015年12月15日发布《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》的《中国在2015年12月25日》